# The Pizza-Oven:  Cooking with a GPU at home.

> I built a machine with a consumer GPU that keeps me cooking with gas at home.  It was a fun project and although I will spin up a Colab instance or other "hosted" solutions from time-to-time this is my everyday _oven_ for cooking up data into something tasty.



## Outline

1. What I wanted to do, and why.
2. What my constraints:  \\$<1000
3. next??  At some point I will add some security and configure my ATT router to allow my to cook in my pizza oven from wherever I may be.


Basically I set up the machine to run linux and act mostly as a jupyter notebook server which i connect thorugh standard SSH tunnels (though sometimes I use it via VSCode instead of a notebook).  I can use my laptop wherever I may be in the house, or use my desktop machine in my office (the Pizza-Oven lives in the freezing cold garage).



## Description

Use these commands to run a jupyter server on a remote server and access it locally in a browser.  You must have SSH access to the browser.  You can use any port you want, but Ill use "8888" for the "server port" and 8889 for the port on the machine I'm connecting from.  E.g. on my laptop I type "localhost:8888" in my browser.  (Firefox please)

[cite ]
**Do not forget to change** `username@server` **to the correct value!**

## TLDR

| where  | using    | command                                          |
| ------ | -------- | ------------------------------------------------ |
| server | terminal | `jupyter notebook --no-browser --port=8888`      |
| local  | terminal | `ssh -v -NL 8888:localhost:8888 username@server` |
| local  | browser  | `localhost:8888`                                 |

## Server

``` bash
screen -R JUPYT
conda activate fastai
jupyter notebook --no-browser --port=8888

```

This starts the jupyter notebook.  Keep the terminal open or run it in the background using screen, tmux, nohup, etc.

```bash
jupyter notebook --no-browser --port=8888
```

For example, you can start it in the background with screen.

```bash
screen -d -m -S JUPYTER jupyter notebook --no-browser --port=8888
```

You can reattach to the screen session if need be.  Use `Ctrl+a+d` to detach.

```bash
screen -r JUPYTER
```

You can stop the screen session altogether, killing the jupyter server.

```bash
screen -ls
screen -X -S JUPYTER quit
```

## Local

I typically use a zsh (on a mac) as my terminal.  So I created a little function in my .zshrc 

This creates an SSH tunnel.  It makes localhost:8888 point to the server's 8888 port.  Fill in the correct `username@server`!



```bash

function jpt(){
    # Fires-up a Jupyter notebook by supplying a specific port
    jupyter notebook --no-browser --port=$1
} # this runs on the GPU machine
# you would type jpt 8890 on pizzapie (GPU)


function remote_jpt(){
    ssh -N -f -L localhost:$2":localhost:"$1 $desktop_ip
    # ssh -N -L local-address:local-port:remote-address:remote-port username:remote-user@remote-host
}  # type jptt 8890 8888 on pizzadashit (x1e)

```
"desktop_ip" is the name of the remote

"-N" is saying don't execute any commands after the "tunnel" is established. That's a little extra security.

"-L" is indicating that we want a Local port forward to the remote machine (-R goes the other way).
   
"local-address" is optional, but good to use, since it restricts what system can access the port.
"local-port" is the forwarded port

```bash

function jpt_tunnel(){
    # "-N" is saying don't execute any commands after the "tunnel" is established. That's a little extra security.
    # "-L" is indicating that we want a Local port forward to the remote machine (-R goes the other way).
    # "local-address" is optional, but good to use, since it restricts what system can access the port.
    # "local-port" is the forwarded port
    screen -d -m -S TUNNEL ssh -N -f -L localhost:$2":localhost:"$1 $desktop_ip
    # ssh -N -L local-address:local-port:remote-address:remote-port username:remote-user@remote-host
}  # type jptt 8890 8888 on pizzadashit (x1e)


```bash
# USE THIS !!
function find_remote_jpt(){
    # find the tunnel process on port ARG
    ps aux | grep localhost:$1
}
```



this is a copy of what runs on the server
```bash

# PROBAABLY NEVER USE THIS
function jpt(){
    # Fires-up a Jupyter notebook by supplying a specific port
    jupyter notebook --no-browser --port=$1
} # this runs on the GPU machine
# you would type jpt 8890 on pizzapie (GPU)
```


this is the crucial one
```bash
# USE THIS !!
remote_ip=ergonyc@pizzapie.local
function remote_jpt(){
    # "-N" is saying don't execute any commands after the "tunnel" is established. That's a little extra security.
    # "-L" is indicating that we want a Local port forward to the remote machine (-R goes the other way).
    # "local-address" is optional, but good to use, since it restricts what system can access the port.
    # "local-port" is the forwarded port
    ssh -N -f -L localhost:$2":localhost:"$1 $remote_ip
    # ssh -N -L local-address:local-port:remote-address:remote-port username:remote-user@remote-host
}  # type jptt 8890 8888 on pizzadashit (x1e)
```

Not using this... 

```bash
function jpt_tunnel(){
    # this will only work if we don't need a password... need to use keys...
    screen -d -m -S TUNNEL ssh -N -f -L localhost:$2":localhost:"$1 $remote_ip
    # ssh -N -L local-address:local-port:remote-address:remote-port username:remote-user@remote-host
}  # type jptt 8890 8888 on pizzadashit (x1e)


```


```bash
ssh -v -NL 8888:localhost:8888 username@server
```

Again, you can start it in the background with screen if you don't want to leave the terminal window open.

```bash
screen -d -m -S TUNNEL ssh -v -NL 8888:localhost:8888 username@server
```

## Browser

This is the web address you have to open in a browser on the local machine.

```
localhost:8888
```

## Screen

Here are some useful screen commands.  Use screen to run things in the background (like the jupyter notebook).

```
screen -ls                    # see whats running
screen -d -m -S NAME command  # start a screen
screen -X    -S NAME quit     # stop a screen
screen -r       NAME          # attach to a screen
```

## References

* http://www.justinkiggins.com/blog/zero-configuration-remote-jupyter-server/

## SSH server settings


### security

    - turn off passwords.  pass public keys
    - dont use the default SSH port (22). use something else (e.g. 2243)
    - 
    
    
> `vim /etc/ssh/sshd_config`
``` bash
    # we will have sudo for proper logins, so turn off root login
    PermitRootLogin no

    # don't allow plaintext passwords
    PasswordAuthentication no

```


Save and close the file. To implement your changes, restart the SSH daemon.

>`sudo service ssh restart`


> `vim .ssh/config`
``` bash
# aliases to make loging in on the right port easy

Host home_gpu_server
    HostName home_gpu_server.local
    Port 2243
```

references: https://www.digitalocean.com/community/tutorials/ssh-essentials-working-with-ssh-servers-clients-and-keys

    
### router settings


ATT Arris BWG210 modem/router.  ATT won't let me set it up to just use the modem and pass onto my own router (Google Nest with 2x sattelite mesh) so I need to set it up for "bridge mode".  I need to forward the right ports and ips so the Nest router can operate correctly.  

NOTE: its set up to use a DHCP range from the Arris which from which the Nest router dynammically allocates IPs to devices on my network.

I need to pass my actuall IP through to my machine with the GPU which i use as a jupyter server.   And also expose the nescessare ports for SSH tunnels into the machine from the outside world. 

Note that once you do this the router can be found at: http://192.168.48.1/  I also left a low-powered 2.4GHz wireless signal running directly from the Arris (wiht an ssid / password I have handy) so I can easily log into change things withough pluggin in with ethernet.  (Because frankly, modern laptops lack an ethernet port, and who wants to dig for a dongle when your network is on the fritz?)

This was the definitive guide I followed, as no "official" guide exists. (this one?)
https://www.reddit.com/r/Ubiquiti/comments/b1x5l6/how_to_properly_configure_the_arris_bgw210_for/


## The Build:
    
    
    - PowermacG5
    - Aorus Pro Wifi $169
    - Intel 9900k ~$450
    - GTX 2070 $400
    - 64GB memory
    - coolers/pwr supply
    
    
    
1. Gutted the machine
2. Cut out the peripherals
3. epoxied the motherboard attachments in place
4. install pop!_os (great built in nvidia drivers)
5. installed cuda / updated drivers...
6. COOK!



side-blog - pics, inspiration, parts list